## WFP-01-01-02 Sentinel-1 coherence timeseries

This application takes a pair of Sentinel-1 products and identifies and generates the coherence

### <a name="quicklink">Quick link

* [Objective](#objective)
* [Test Site](#test-site)
* [Context](#context)
* [Applicability](#applicability)
* [Data](#data)
* [Service Definition](#service)
* [Parameter Definition](#parameter)
* [Runtime Parameter Definition](#runtime)
* [Workflow](#workflow)
* [Strengths and Limitations](#strengths-limitations) 
* [License](#license)

As a data processor developer, I want to implement, and package an algorithm processing a pair of S1 SAR SLC datasets using the SNAP toolbox notebook archetype with the following processing steps:

A. S1 SAR processing per image:

* Application of orbit file (should wait for the orbit file a couple of days, since for coherence this is important)
* TOPS slice assembly (if necessary)
* TOPS split (if necessary)

B. For image pair:

* TOPS coregistration
* Coherence estimation (Given set of images from same orbit,  {t1, t2, t3, ... , tn}, two temporally adjacent images would * constitute an image pair, with the first one being the master. So the pairs would be: {t1 - t2, t2 - t3, ... , tn-1 - tn}.)
* TOPS deburst
* Multi-looking
* Terrain correction


TOPS --> Terrain Observation by Prograssive Scans

TOPS Slice assembly-->assemble subswaths back together

TOPS split -->split into subswaths



### <a name="service">Service definition

In [1]:
service = dict([('title', 'WFP-01-01-02 Sentinel-1 coherence'),
                ('abstract', 'WFP-01-01-02 Sentinel-1 coherence'),
                ('id', 'ewf-wfp-01-01-02')])

In [3]:
swaths = dict([('id', 'swaths'),
               ('value', 'IW3'),
               ('title', 'Sentinel-1 sub-swaths'),
               ('abstract', 'Sentinel-1 sub-swaths')])

In [4]:
subswaths = list(swaths['value'].split(',')) 

In [5]:
subswaths

['IW3']

In [6]:
polarisation = dict([('id', 'polarisation'),
                     ('value', 'VV'),
                     ('title', 'Polarisation'),
                     ('abstract', 'Polarisation')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

These are the Sentinel-1 product identifiers

In [7]:

# input_identifiers = ('S1A_IW_SLC__1SDV_20180701T033302_20180701T033332_022595_0272AB_04FA',
#                      'S1B_IW_SLC__1SDV_20180625T033210_20180625T033237_011524_0152DA_9FCC',
#                      'S1B_IW_SLC__1SDV_20180707T033210_20180707T033237_011699_01584C_0A8E')
# input_identifiers = ('S1B_IW_SLC__1SDV_20180707T033235_20180707T033303_011699_01584C_D2BE',
#                      'S1B_IW_SLC__1SDV_20180707T033210_20180707T033237_011699_01584C_0A8E',
#                      'S1A_IW_SLC__1SDV_20180713T033303_20180713T033332_022770_0277E0_8A9E')
input_identifiers = ('S1A_IW_SLC__1SDV_20170105T035000_20170105T035030_014691_017E74_0110', 
                     'S1B_IW_SLC__1SDV_20170111T034920_20170111T034947_003795_006862_116E')

**Input references**

These are the Sentinel-1 catalogue references

In [8]:
# input_references = ('https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20180701T033302_20180701T033332_022595_0272AB_04FA',
#                     'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1B_IW_SLC__1SDV_20180625T033210_20180625T033237_011524_0152DA_9FCC',
#                     'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1B_IW_SLC__1SDV_20180707T033210_20180707T033237_011699_01584C_0A8E')


# input_references = ('https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1B_IW_SLC__1SDV_20180707T033235_20180707T033303_011699_01584C_D2BE',
#  'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1B_IW_SLC__1SDV_20180707T033210_20180707T033237_011699_01584C_0A8E',
#  'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20180713T033303_20180713T033332_022770_0277E0_8A9E')
input_references = ('https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20170105T035000_20170105T035030_014691_017E74_0110',
'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1B_IW_SLC__1SDV_20170111T034920_20170111T034947_003795_006862_116E')


**Data path**

This path defines where the data is staged-in. 

In [9]:
data_path = '/workspace/data/test'

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [10]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())
import ellip_snap_helpers

from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap

import dateutil.parser as parser
import gc
from datetime import datetime

import gzip
import shutil

import gdal
import osr

import lxml.etree as etree

from shapely.wkt import loads
import numpy as np

from shapely.geometry import box

import warnings
warnings.filterwarnings("ignore")

import glob

sys.path.append('/opt/anaconda/bin/')

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors



import cioppy
ciop = cioppy.Cioppy()

## Read the products

### check if all the products have the same track number

In [11]:
product_TR=[None]*len(input_references)

for index,product_ref in enumerate(input_references):
    Result_Prod=ciop.search(end_point = product_ref,params =[],output_fields='startdate,track',model='EOP')
    product_TR[index]=Result_Prod[0]['track']
    if index==0:
        Slave_Date=Result_Prod[0]['startdate'][:10]
    elif Result_Prod[0]['startdate'][:10] > Slave_Date:
        Slave_Date=Result_Prod[0]['startdate'][:10]
if not all(x == product_TR[0] for x in product_TR):
     raise ValueError('Not all products pertain the same track !')

In [12]:
 Slave_Date

'2017-01-11'

#### Read the products

In [13]:
s1meta = "manifest.safe"

slave_date = ''

slave_products = []
master_products = []

slave_prefix = []
master_prefix = []

dates = []

for index, identifier in enumerate(input_identifiers):

    
    s1_zip_file = os.path.join(data_path, identifier + '.zip') 
    s1_meta_file = os.path.join(data_path, identifier, identifier + '.SAFE', 'manifest.safe') 

    if os.path.isfile(s1_zip_file):
        s1prd = s1_zip_file
    elif os.path.isfile(s1_meta_file):
        s1prd = s1_meta_file

    #print identifier, s1prd
    reader = ProductIO.getProductReader("SENTINEL-1")
    product = reader.readProductNodes(s1prd, None)
    
    
    width = product.getSceneRasterWidth()
    height = product.getSceneRasterHeight()
    name = product.getName()
    start_date = parser.parse(product.getStartTime().toString()).isoformat()
    
    dates.append(start_date[:19])

    if start_date[:10] == Slave_Date:
            slave_products.append(s1prd)
            print("\nProduct: %s, %d x %d pixels of %s assigned as slave" % (name, width, height, start_date))
            slave_prefix.append(identifier.split('_')[-1]) 
            slave_data_take = identifier.split('_')[-2]
    else:
            master_products.append(s1prd)
            print("\nProduct: %s, %d x %d pixels of %s assigned as master" % (name, width, height, start_date))
            master_data_take = identifier.split('_')[-2]  
            master_prefix.append(identifier.split('_')[-1]) 

            
            
output_name = 'S1_COH_%s_%s_%s_%s' % (parser.parse(max(dates)).strftime('%Y%m%d%H%M%S'),
                                                     parser.parse(min(dates)).strftime('%Y%m%d%H%M%S'),
                                                     '_'.join(slave_prefix),
                                                     '_'.join(master_prefix))

print("\nco-registered OUTPUT Img name is %s"%output_name)



Product: S1A_IW_SLC__1SDV_20170105T035000_20170105T035030_014691_017E74_0110, 68485 x 15040 pixels of 2017-01-05T03:50:00.278426 assigned as master

Product: S1B_IW_SLC__1SDV_20170111T034920_20170111T034947_003795_006862_116E, 68909 x 13563 pixels of 2017-01-11T03:49:20.125922 assigned as slave

co-registered OUTPUT Img name is S1_COH_20170111034920_20170105035000_116E_0110


In [14]:
mygraph = ellip_snap_helpers.GraphProcessor()

#### Read and if need assemble the products

In [15]:
operator = 'Read'

node_id = 'Read'

source_node_id = ''

In [16]:
# if len(slave_products) > 1:
  
#     slave_read_nodes = []
    
#     # Read 
#     for index, slave_identifier in enumerate(slave_products):
        
#         operator = 'Read'
        
#         parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
        
#         node_id = 'Read-S(%s)' % index
        
#         source_node_id = ''
        
#         parameters['file'] = slave_identifier
                
#         mygraph.add_node(node_id, operator, parameters, source_node_id)
    
#         slave_read_nodes.append(node_id)
    
    
#     source_nodes_id = slave_read_nodes
        
#     operator = 'SliceAssembly'
       
#     node_id = 'SliceAssembly-S'
    
#     parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    
#     parameters['selectedPolarisations'] = polarisation['value']
    
#     mygraph.add_node(node_id, operator, parameters, source_nodes_id)

#     source_slave_orbit = node_id
    
# else:
operator = 'Read'
        
parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
        
node_id = 'Read-S'
        
source_node_id = ''
        
parameters['file'] = slave_products[0]
        
mygraph.add_node(node_id, operator, parameters, source_node_id)
    
source_slave_orbit = node_id

In [17]:
if len(master_products) > 1:
  
    master_read_nodes = []
    
    # Read 
    for index, master_identifer in enumerate(master_products):
        
        operator = 'Read'
        
        parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
        
        node_id = 'Read-M(%s)' % index
        
        source_node_id = ''
        
        parameters['file'] = master_identifer
        
        mygraph.add_node(node_id, operator, parameters, source_node_id)
    
        master_read_nodes.append(node_id)
    
    
    source_nodes_id = master_read_nodes
        
    operator = 'SliceAssembly'
       
    node_id = 'SliceAssembly-M'
    
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    
    parameters['selectedPolarisations'] = polarisation['value']
    
    mygraph.add_node(node_id, operator, parameters, source_nodes_id)

    source_master_orbit = node_id
    
else:
    
    operator = 'Read'
        
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
        
    node_id = 'Read-M'
        
    source_node_id = ''
        
    parameters['file'] = master_products[0]
        
    mygraph.add_node(node_id, operator, parameters, source_node_id)
    
source_master_orbit = node_id

In [18]:
mygraph.view_graph()


<graph>
  <version>1.0</version>
  <node id="Read-S">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/test/S1B_IW_SLC__1SDV_20170111T034920_20170111T034947_003795_006862_116E/S1B_IW_SLC__1SDV_20170111T034920_20170111T034947_003795_006862_116E.SAFE/manifest.safe</file>
    </parameters>
  </node>
  <node id="Read-M">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/test/S1A_IW_SLC__1SDV_20170105T035000_20170105T035030_014691_017E74_0110.zip</file>
    </parameters>
  </node>
</graph>



### Apply orbit file

In [19]:
operator = 'Apply-Orbit-File'

node_id = 'Apply-Orbit-File-S' 

source_node_id = source_slave_orbit

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

In [20]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

In [21]:
operator = 'Apply-Orbit-File'

node_id = 'Apply-Orbit-File-M' 

source_node_id = source_master_orbit

In [22]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

In [23]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read-S">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/test/S1B_IW_SLC__1SDV_20170111T034920_20170111T034947_003795_006862_116E/S1B_IW_SLC__1SDV_20170111T034920_20170111T034947_003795_006862_116E.SAFE/manifest.safe</file>
    </parameters>
  </node>
  <node id="Read-M">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/test/S1A_IW_SLC__1SDV_20170105T035000_20170105T035030_014691_017E74_0110.zip</file>
    </parameters>
  </node>
  <node id="Apply-Orbit-File-S">
    <operator>Apply-Orbit-File</operator>
    <sources>
      <sourceProduct refid="Read-S"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <polyDegree>3</polyDegree>
      <orbitType>Sentinel Precise (Auto Download)</orbitType>
   

### Split

In [24]:
slave_split_nodes = []

operator = 'TOPSAR-Split'

source_node_id = 'Apply-Orbit-File-S'

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

for subswath in subswaths:
    
    node_id = 'TOPSAR-Split-S-(%s)' % subswath
    
    slave_split_nodes.append(node_id)
    
    parameters['subswath'] = subswath
    parameters['selectedPolarisations'] = polarisation['value']

    mygraph.add_node(node_id, operator, parameters, source_node_id)



In [25]:
master_split_nodes = []

operator = 'TOPSAR-Split'

source_node_id = 'Apply-Orbit-File-M'

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

for subswath in subswaths:
    
    node_id = 'TOPSAR-Split-M-(%s)' % subswath
    
    master_split_nodes.append(node_id)
    
    parameters['subswath'] = subswath
    parameters['selectedPolarisations'] = polarisation['value']

    mygraph.add_node(node_id, operator, parameters, source_node_id)


### Back-Geocoding

In [26]:
operator = 'Back-Geocoding'


parameters = ellip_snap_helpers.get_operator_default_parameters(operator)


In [27]:

backgeo_nodes = []

for index, subswath in enumerate(subswaths):  
  
    node_id = 'Back-Geocoding-(%s)' % subswath
    
    backgeo_nodes.append(node_id)
    
    mygraph.add_node(node_id, operator, parameters, [slave_split_nodes[index], master_split_nodes[index]])





#### Coherence

In [28]:
operator = 'Coherence'

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)


In [29]:
coherence_nodes = []

for index, subswath in enumerate(subswaths):  
  
    node_id = 'Coherence-(%s)' % subswath
    
    coherence_nodes.append(node_id)
    
    mygraph.add_node(node_id, operator, parameters, backgeo_nodes[index])

In [30]:
operator = 'TOPSAR-Deburst'

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)


In [31]:
deburst_nodes = []

for index, subswath in enumerate(subswaths):  
 
    node_id = 'TOPSAR-Deburst-(%s)' % subswath
    
    deburst_nodes.append(node_id)
    
    mygraph.add_node(node_id, operator, parameters, coherence_nodes[index])


In [32]:
if len(subswaths)>1:

    operator = 'TOPSAR-Merge'

    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    parameters['selectedPolarisations'] = polarisation['value']

    node_id = 'TOPSAR-Merge'

    mygraph.add_node(node_id, operator, parameters, deburst_nodes)
    next_source_node_id='TOPSAR-Merge'
else: 
    next_source_node_id='TOPSAR-Merge'

In [33]:
operator = 'Terrain-Correction'

#source_node_id = 'TOPSAR-Merge'

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
parameters['outputComplex'] = 'true'
parameters['pixelSpacingInMeter'] = '10.0'

node_id = 'Terrain-Correction'

#mygraph.add_node(node_id, operator, parameters, source_node_id)
mygraph.add_node(node_id, operator, parameters, deburst_nodes)

In [34]:
operator = 'Write'

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
parameters['file'] = output_name
parameters['formatName'] = 'GeoTIFF-BigTiff'

node_id = 'Write'

source_node_id = 'Terrain-Correction'

mygraph.add_node(node_id, operator, parameters, source_node_id)

In [35]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read-S">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/test/S1B_IW_SLC__1SDV_20170111T034920_20170111T034947_003795_006862_116E/S1B_IW_SLC__1SDV_20170111T034920_20170111T034947_003795_006862_116E.SAFE/manifest.safe</file>
    </parameters>
  </node>
  <node id="Read-M">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/test/S1A_IW_SLC__1SDV_20170105T035000_20170105T035030_014691_017E74_0110.zip</file>
    </parameters>
  </node>
  <node id="Apply-Orbit-File-S">
    <operator>Apply-Orbit-File</operator>
    <sources>
      <sourceProduct refid="Read-S"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <polyDegree>3</polyDegree>
      <orbitType>Sentinel Precise (Auto Download)</orbitType>
   

In [36]:
mygraph.run()

Processing the graph
Process PID: 5733
Executing processing graph
Master: 11Jan2017
Slave: 11Jan2017 prep baseline: 0.0 temp baseline: 0.0
Slave: 05Jan2017 prep baseline: -75.62477 temp baseline: 5.999513

Master: 05Jan2017
Slave: 11Jan2017 prep baseline: 75.42546 temp baseline: -5.999513
Slave: 05Jan2017 prep baseline: 0.0 temp baseline: 0.0

Please select split sub-swaths of the same Sentinel-1 products
 done.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.hsqldb.persist.Logger: dataFileCache open start

Error: [NodeId: TOPSAR-Merge] Please select split sub-swaths of the same Sentinel-1 products
-- org.jblas INFO Deleting /tmp/jblas4277208155257815930/libjblas.so
-- org.jblas INFO Deleting /tmp/jblas4277208155257815930/libjblas_arch_flavor.so
-- org.jblas INFO Deleting /tmp/jblas4277208155257815930

Done.


#### Results metadata

In [37]:
def eop_metadata(metadata):

    opt = 'http://www.opengis.net/opt/2.1'
    om  = 'http://www.opengis.net/om/2.0'
    gml = 'http://www.opengis.net/gml/3.2'
    eop = 'http://www.opengis.net/eop/2.1'
    sar = 'http://www.opengis.net/sar/2.1'
    
    root = etree.Element('{%s}EarthObservation' % sar)

    phenomenon_time = etree.SubElement(root, '{%s}phenomenonTime' % om)

    time_period = etree.SubElement(phenomenon_time, '{%s}TimePeriod' % gml)

    begin_position = etree.SubElement(time_period, '{%s}beginPosition'  % gml)

    end_position = etree.SubElement(time_period, '{%s}endPosition'  % gml)

    procedure = etree.SubElement(root, '{%s}procedure' % om)

    earth_observation_equipment = etree.SubElement(procedure, '{%s}EarthObservationEquipment' % eop)

    acquisition_parameters = etree.SubElement(earth_observation_equipment, '{%s}acquisitionParameters' % eop)

    acquisition = etree.SubElement(acquisition_parameters, '{%s}Acquisition' % sar)

    orbit_number = etree.SubElement(acquisition, '{%s}orbitNumber' % eop)

    wrs_longitude_grid = etree.SubElement(acquisition, '{%s}wrsLongitudeGrid' % eop)

    polarisation_channels = etree.SubElement(acquisition, '{%s}polarisationChannels' % eop)
    
    feature_of_interest = etree.SubElement(root, '{%s}featureOfInterest' % om)
    footprint = etree.SubElement(feature_of_interest, '{%s}Footprint' % eop)
    multi_extentOf = etree.SubElement(footprint, '{%s}multiExtentOf' % eop)
    multi_surface = etree.SubElement(multi_extentOf, '{%s}MultiSurface' % gml)
    surface_members = etree.SubElement(multi_surface, '{%s}surfaceMembers' % gml)
    polygon = etree.SubElement(surface_members, '{%s}Polygon' % gml)    
    exterior = etree.SubElement(polygon, '{%s}exterior' % gml)  
    linear_ring = etree.SubElement(exterior, '{%s}LinearRing' % gml) 
    poslist = etree.SubElement(linear_ring, '{%s}posList' % gml) 


    result = etree.SubElement(root, '{%s}result' % om)
    earth_observation_result = etree.SubElement(result, '{%s}EarthObservationResult' % opt)
    cloud_cover_percentage = etree.SubElement(earth_observation_result, '{%s}cloudCoverPercentage' % opt)
    
    metadata_property = etree.SubElement(root, '{%s}metaDataProperty' % eop)
    earth_observation_metadata = etree.SubElement(metadata_property, '{%s}EarthObservationMetaData' % eop)
    identifier = etree.SubElement(earth_observation_metadata, '{%s}identifier' % eop)
    
    begin_position.text = metadata['startdate']
    end_position.text = metadata['enddate']
    
    coords = np.asarray([t[::-1] for t in list(loads(metadata['wkt']).exterior.coords)]).tolist()
 
    pos_list = ''
    for elem in coords:
        pos_list += ' '.join(str(e) for e in elem) + ' '   

    poslist.attrib['count'] = str(len(coords))
    poslist.text = pos_list
    
    
    identifier.text = metadata['identifier'] 

    return etree.tostring(root, pretty_print=True)

#### Get the result WKT


In [38]:
src = gdal.Open(output_name + '.tif')
ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()

max_x = ulx + (src.RasterXSize * xres)
min_y = uly + (src.RasterYSize * yres)
min_x = ulx 
max_y = uly

source = osr.SpatialReference()
source.ImportFromWkt(src.GetProjection())

target = osr.SpatialReference()
target.ImportFromEPSG(4326)

transform = osr.CoordinateTransformation(source, target)

result_wkt = box(transform.TransformPoint(min_x, min_y)[0],
        transform.TransformPoint(min_x, min_y)[1],
        transform.TransformPoint(max_x, max_y)[0],
        transform.TransformPoint(max_x, max_y)[1]).wkt

AttributeError: 'NoneType' object has no attribute 'GetGeoTransform'

#### Create the EOP XML metadata file

In [36]:
import json
eop_metadata = dict()

eop_metadata['wkt'] = result_wkt
eop_metadata['startdate'] = parser.parse(min(dates)).strftime('%Y-%m-%dT%H:%M:%S')
eop_metadata['enddate'] = parser.parse(max(dates)).strftime('%Y-%m-%dT%H:%M:%S')

eop_xml = output_name + '.xml'
with open(eop_xml, 'wb') as file:
    file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    file.write(json.dumps(eop_metadata))

#### Create the properties file for the reproducibility notebooks

In [37]:
for properties_file in ['result', 'stage-in']:

    if properties_file == 'result':
        title = 'Reproducibility notebook used for generating %s' % output_name
    else: 
        title = 'Reproducibility stage-in notebook for Sentinel-1 data for generating %s' % output_name
        
    with open(properties_file + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (parser.parse(min(dates)).strftime('%Y-%m-%dT%H:%M:%S'), parser.parse(max(dates)).strftime('%Y-%m-%dT%H:%M:%S')))      
        file.write('geometry=%s' % (result_wkt))

### Compress the file

In [38]:
with open(output_name + '.tif', 'rb') as f_in, gzip.open(output_name + '.gz', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)
    
os.remove(output_name + '.tif')

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.